# Streamlit으로 인공지능 앱 개발하기

## Streamlit
- Streamlit은 Streamlit 사가 개발한 웹 앱 프레임워크로 데이터 분석 및 기계 학습 코드를 통합한 웹 앱을 간단하게 구축하고 공개할 수 있음
- Streamlit의 사이트
> URL https://streamlit.io
---
- Streamlit을 사용해 데이터 분석 결과를 간단히 웹 페이지에 표시 할 수 있음
  - 예를 들어 Pandas의 DataFrame을 표로 표시하거나 matplotlib 등으로 작성한 그래프를 삽입 가능
  - 간결하고 사용하기 쉬운 UI를 구현할 수 있고 다양한 타입의 앱에 대응 가능
- 손쉬운 만틈 반대로 복잡한 앱 개발은 어렵지만 인공지능 모델의 데모를 만들거나 데이터 분석의 결과를 쉽게 멤버들과 공유하고 싶을 떄 매우 편리함
- 또한 Streamlit Cloud라는 서비스를 사용하면 GitHub을 경유해 구축한 인공지능 앱을 쉽게 공개할 수 있음
  - HTML 코드 작성 필요 X
- 개발 예시는 공식 사이트의 갤러리에서 확인 가능
> URL https://streamlit.io/gallery
  - BERT에 의한 자연어 처리 앱 및 GAN에 의한 얼굴 이미지 생성 앱, 지도를 사용한 앱 등에서 다양한 타입의 인공지능 앱이 소개되어 있음
  

## Streamlit을 이용한 인공지능 앱 개발
- 이번 장에선 패션 아이템을 식별하는 인공지능 웹 앱 구축
- 패션 아이템의 이미지는 로컬에서 업로드하거나 카메라로 촬영해서 입수함
- 그런 다음 이 이미지를 입력해서 훈련한 모델로 예측이 이뤄짐
- 예측 결과는 화면에 문장 및 원 그래프로 표시됨

### 이번 장에서는 다음과 같은 흐름으로 인공지능 앱을 구축하고 공개함
1. CNN 모델의 훈련
2. Streamlit으로 웹 앱을 구축
3. GitHub 경유로 Streamlit Cloud에 앱을 배포
---
- 먼저 PyTorch에서 CNN 모델을 구축하고 패션 아이템의 이미지를 훈련 데이터로서 사용하여 모델을 훈련함
- 글고 훈련한 모델을 사용하는 웹 앱을 프레임워크 Streamlit을 사용하여 구축함
- 구축한 앱은 GitHub의 저장소에 업로드하고 Streamlit Cloud와 연계하여 클라우드에 공개함. 이때 URL이 발행되므로 공유해서 많은 사람이 앱을 사용할 수 있음

# 모델 구축과 훈련
- Google Colaboratory에서 이미지 식별용의 모델을 구축하고 훈련함
- 이번은 Fashion-MNIST를 훈련 데이터로 사용하여 패션 아이템을 식별할 수 있도록 보델을 훈련함
- 훈련한 모델은 저장하고 다운로드함

## 훈련 데이터 읽어 들이기와 DAtaLoader의 설정
- 훈련 데이터로서 RNN을 이용한 이미지 생성에서 설명한 데이터셋 Fashion-MNIST를 읽어들임
  - Fashion-MNIST에는 10개 카테고리, 합계 70000장의 패션 아이템 이미지가 포함되어 있음
  - 이미지는 $28 \times 28$ 픽셀
- DataLoader를 데이터 확장과 함께 설정함
- 이번에는 배경의 밝기 변동에 대해서도 잘 대응하도록 색 반전도 시행함
---
* Fashion-MNIST의 원본 데이터는 MIT 라이선스임. MIT 라이선스는 라이선스 표기가 필요하지만 비교적 자유롭게 사용 가능.
* 훈련 데이터를 서비스에 이용할 때는 그 데이터의 라이선스에 주의를 기울여야 함
- fashion-mnist
> URL https://github.com/zalandoresearch/fashion-mnist


In [ ]:
# 훈련 데이터 읽어들이기와 DataLoader의 설정
from torchvision.datasets import FashionMNIST
import torchvision.transforms as transforms
from torch.utils.data import DataLoader

affine = transforms.RandomAffine((-30, 30), # 회전
                                 scale = (0.8, 1.2), # 확대와 축소
                                 translate = (0.5, 0.5)) # 이동

flip = transforms.RandomHorizontalFlip(p = 0.5) # 좌우 반전
invert = transforms.RandomInvert(p = 0.5) # 색의 반전
to_tensor = transforms.ToTensor()
normalize = transforms.Normalize((0.0), (1.0)) # 평균값을 0, 표준편차를 1로
erase = transforms.RandomErasing(p = 0.5) # 일부를 소거

transform_train = transforms.Compose([affine, flip, invert, to_tensor, normalize, erase])
transform_test = transforms.Compose([to_tensor, normalize])
fashion_train = FashionMNIST("./data", train = True, download = True, transform = transform_train)
fashion_test = FashionMNIST("./data", train = False, download = True, transform = transform_test)

# DataLoader의 설정
batch_size = 64
train_loader = DataLoader(fashion_train, batch_size = batch_size, shuffle = True)
test_loader = DataLoader(fashion_test, batch_size = batch_size, shuffle = False)


## 모델 구축
- nn.Module() 클래스를 상속받은 클래스로서 CNN 모델을 구축함
- 이번엔 <U>배치 정규화(Batch Normalization)을 위한 층인 **nn.BatchNorm2d()** 클래스</U>를 추가함
- 배치 정규화는 네트워크 도중에 데이터를 평균 0, 표준편차 1로 변환하여 데이터 분포의 편향을 방지함
- 이로 인해 학습이 안정화되고 속도가 빨라짐
- 또한, 배치 정규화의 층에는 학습 파라미터가 있으므로 층을 다시 사용할 수 없음

In [ ]:
# 이미지 인식 모델의 구축
import torch.nn as nn

class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 8, 3)
    self.conv2 = nn.Conv2d(8, 16, 3)
    self.bn1 = nn.BatchNorm2d(16)
    self.conv3 = nn.Conv2d(16, 32, 3)
    self.conv4 = nn.Conv2d(32, 64, 3)
    self.bn2 = nn.BatchNorm2d(64)

    self.pool = nn.MaxPool2d(2, 2)
    self.relu = nn.ReLU()

    self.fc1 = nn.Linear(64*4*4, 256)
    self.dropout = nn.Dropout(p = 0.5)
    self.fc2 = nn.Linear(256, 10)

  def forward(self, x):
    x = self.relu(self.conv1(x))
    x = self.relu(self.bn1(self.conv2(x)))
    x = self.pool(x)
    x= self.relu(self.conv3(x))
    x = self.relu(self.bn2(self.conv4(x)))
    x = self.pool(x)
    x = x.view(-1, 64*4*4)
    x = self.relu(self.fc1(x))
    x = self.dropout(x)
    x = self.fc2(x)
    return x

net = Net()
net.cuda() # GPU 대응
print(net)

## 학습
- 이미지 인식 모델을 훈련함
- DataLoader를 사용하여 미니 배치를 꺼내 훈련 및 평가를 실시함

In [ ]:
# 이미지 인식 모델의 훈련
from torch import optim

# 교차 엔트로피 오차 함수
loss_fnc = nn.CrossEntropyLoss()

# 최적화 알고리즘
optimizer = optim.Adam(net.parameters())

# 손실 로그
record_loss_train = []
record_loss_test = []

# 학습
for i in range(30): # 30 에포크 학습
  net.train() # 훈련 모드
  loss_train = 0
  for j, (x, t) in enumerate(train_loader): # 미니 배치 (x, t)를 꺼낸다
    x, t = x.cuda(), t.cuda() # GPU 대응
    y = net(x)
    loss = loss_fnc(y, t)
    loss_train += loss.item()
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  loss_train /= j+1
  record_loss_train.append(loss_train)

  net.eval() # 평가 모드
  loss_test = 0
  for j, (x, t) in enumerate(test_loader): # 미니 배치 (x, t)를 꺼낸다
    x, t = x.cuda(), t.cuda() # GPU 대응
    y = net(x)
    loss =loss_fnc(y, t)
    loss_test += loss.item()
  loss_test /= j+1
  record_loss_test.append(loss_test)

  if i%1 == 0 :
    print("Epoch: ", i, "Loss_Train: ", loss_train, "Loss_test: ", loss_test)

## 오차 추이
- 훈련 데이터와 테스트 데이터 각각 오차 추이를 그래프로 표시함

In [ ]:
# 오차 추이
import matplotlib.pyplot as plt

plt.plot(range(len(record_loss_train)), record_loss_train, label = "Train")
plt.plot(range(len(record_loss_test)), record_loss_test, label = "Test")
plt.legend()

plt.xlabel("Epochs")
plt.ylabel("Errors")
plt.show()

## 정답률
- 모델의 성능을 파악하기 위해서 테스트 데이터를 사용해 정답률을 측정함

In [ ]:
# 정답률의 계산
correct = 0
total = 0
net.eval() # 평가 모드
for i, (x, t) in enumerate(test_loader) :
  x, t = x.cuda(), t.cuda() # GPU 대응
  y = net(x)
  correct += (y.argmax(1) == t).sum().item()
  total += len(x)

print("정답률: ", str(correct/total * 100) + "%")

## 모델 저장
- <U>훈련한 모델의 파라미터를 **state_dict()** 메서드</U>에 의해 취득하고 저장함
- 다음 코드는 state_dict() 메서드의 내용을 표시한 후 model_cnn.pth라는 파일명으로 저장함

In [ ]:
# 모델 저장
import torch

# state_dict()의 표시
for key in net.state_dict():
  print(key, ": ", net.state_dict()[key].size())

# 저장
torch.save(net.state_dict(), "model_cnn.pth")

## 훈련한 파라미터의 다운로드
- 훈련한 모델의 파라미터 model_cnn.pth를 로컬 환경으로 다운로드 해둠
- 로컬 환경에 다운로드 해 둔 뒤에, 이 파일을 앱 구축에 이용함

# 이미지 인식 앱의 구축
- Streamlit을 사용하여 이미지를 인식하는 앱을 만듦
- 프레임워크로는 PyTorch를 사용하여 원본 CNN 모델을 읽어들여 사용함
- 이번은 Google Colaboratory에서 다음 2개의 파일을 만듦
  - model.py
  - app.py
- app.py가 앱의 본체이고, model.py는 훈련한 모델을 하용해 예측을 실시하는 파일임
- 이것들을 동작시키기 위해서는 이전에 만든 훈련한 파라미터 model_cnn.pth를 업로드 해야 함
- 이번엔 **ngrok**이라는 툴을 사용해 앱의 동작을 확인함

## ngrok의 설정
- **ngrok**은 로컬 서버를 외부 공개할 수 있는 톨
- 이번은 Google Colaboratory 서버에서 이 ngrok을 사용하여 앱을 공개하고 동작을 확인함
- Google Colaboratory에서 ngrok을 사용하려면 ngrok 사이트에 등록하여 **Authtoken**을 취득해야 함
- ngrok 웹사이트
> URL https://ngrok.com
- 로그인 후 인증이 완료되면 ngrok의 대시보드에 도달할 수 있음
- 여기서 왼쪽 메뉴의 ** Your Authtoken**을 선택하면 authtoken이 표시되는데 이것을 Copy 버튼을 클릭하여 복사해 둠
- 이것은 이후 Authtoken의 설정에서 노트북의 Your Authtoken 부분에 붙여넣게 됨

## 라이브러리 설치
- Streamlit 및 앱의 동작 확인에 사용하는 ngrok을 설치함

In [2]:
# 필요한 라이브러리의 설치
%pip install streamlit==1.8.1 --quiet
%pip install pyngrok==4.1.1 --quiet

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
# ngrok 다운로드
!wget https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz -O ngrok.tgz

# 압축 해제
!tar -xvzf ngrok.tgz

# ngrok 실행 파일을 이동
!mv ngrok /usr/local/bin/


In [ ]:
!ngrok config upgrade

In [ ]:
!ngrok config check

In [1]:
# Streamlit과 ngrok을 import한다
import streamlit as st
from pyngrok import conf, ngrok

TypeError: Descriptors cannot not be created directly.
If this call came from a _pb2.py file, your generated code is out of date and must be regenerated with protoc >= 3.19.0.
If you cannot immediately regenerate your protos, some other possible workarounds are:
 1. Downgrade the protobuf package to 3.20.x or lower.
 2. Set PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION=python (but this will use pure-Python parsing and will be much slower).

More information: https://developers.google.com/protocol-buffers/docs/news/2022-05-06#python-updates

## 훈련한 파라미터를 업로드
- 훈련한 파라미터 model_cnn.pth를 업로드함
- 페이지 왼쪽의 파일의 아이콘을 클릭하고, 표시된 메뉴에서 조금 전에 받아둔 model_cnn.pth를 선택하여 열린 영역으로 드래그&드롭함
  - 이로 인해 model_cnn.pth가 Google Colaboratory의 서버에 업로드되고, 노트북에서 읽어 들일 수 있게 됨

## 모델을 다루는 파일
- 이미지 인식을 훈련한 모델을 읽어 들이고, 예측을 하는 코드를 model.py에 써넣는다.
- 다음 코드의 앞에 있는 **%%writefile**은 매직 커맨드의 일종으로, 지정한 파일에 셀의 내용을 적어 넣음. 이 경우는 이 행 이후의 코드가 파일 model.py에 쓰임
- **predict()** 함수는 <U>인수로 img를 받는데</U> 이것은 PIL(Pillow)의 Image형임.
- 이것은 훈련한 모델에 맞게 흑백으로 변환하고 크기도 변환함
- 나머지는 Tensor로 변환한 다음에 이것을 훈련한 모델에 입력함
- 그리고 예측 결과를 조정하여 반환값으로 함

In [ ]:
# 모델을 다루는 파일 model.py

%%writefile model.py
# 이하를 model.py에 써넣기
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models, transforms
from PIL import Image

classes_kr = ["셔츠/톱", "바지", "풀오버", "드레스", "코드", "샌들", "와이셔츠", "스니커즈", "가방", "앵클부츠"]
classes_en = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat", "Sandal", "Shirt", "Sneaker", "Bag", "Ankel boot"]
n_class = len(classes_kr)
img_size = 28

# 이미지 인식의 모델
class Net(nn.Module):
  def __init__(self):
    super().__init__()
    self.conv1 = nn.Conv2d(1, 8, 3)
    self.conv2 = nn.Conv2d(8, 16, 3)
    self.bn1 = nn.BatchNorm2d(16)
    self.conv3 = nn.Conv2d(16, 32, 3)
    self.conv4 = nn.Conv2d(32, 64, 3)
    self.bn2 = nn.BatchNorm2d(64)

    self.pool = nn.MaxPool2d(2, 2)
    self.relu = nn.ReLU()

    self.fc1 = nn.Linear(64*4*4, 256)
    self.dropout = nn.Dropout(p=0.5)
    self.fc2 = nn.Linear(256, 10)

  def forward(self, x):
    x = self.relu(self.conv1(x))
    x = self.relu(self.bn1(self.conv2(x)))
    x = self.pool(x)
    x = F.relu(self.conv3(x))
    x = F.relu(self.bn2(self.conv4(x)))
    x = self.pool(x)
    x = x.view(-1, 64*4*4)
    x = F.relu(self.fc1(x))
    x = self.dropout(x)
    x = self.fc2(x)
    return x

net = Net()

# 훈련한 파라미터 읽어 들이기와 설정
net.load_state_dict(torch.load("model_cnn.pth", map_location = torch.device("cpu")))

def predict(img):
  # 모델로 입력
  img = img.convert("L") # 흑백으로 변환
  img = img.resize((img_size, img_size)) # 크기를 변환
  transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.0), (1.0))])
  img = transform(img)
  x = img.reshape(1, 1, img_size, img_size)

  # 예측
  net.eval()
  y = net(x)

  # 결과를 반환한다
  y_prob = torch.nn.functional.softmax(torch.squeeze(y)) # 확률로 나타낸다
  sorted_prob, sorted_indices = torch.sort(y_prob, descending = True) # 내림차순으로 정렬
  return [(classes_kr[idx], classes_en[idx], prob.item()) for idx, prob in zip(sorted_indices, sorted_prob)]

## 앱의 코드
- 이미지 인식 앱 본체의 코드를 **app.py**에 써넣음
- 로컬에서 업로드 혹은 웹 카메라로 촬영한 이미지 파일에 무엇이 찍혀 있는지를 model.py의 predict() 함수를 사용해 판정함

In [ ]:
# 앱 본체의 파일 app.py

%%writefile app.py
# 이하를 app.py에 써넣기
import streamlit as st
import matplotlib.pyplot as plt
from PIL import Image
from model import predict

st.set_option("deprecation.showfileUploaderEncoding", False)

st.sidebar.title("이미지 인식 앱")
st.sidebar.write("원본 이미지 인식 모델을 사용해서 무슨 이미지인지를 판정합니다.")

st.sidear.write("")

img_source = st.sidebar.radio("이미지 소스를 선택해 주세요.", ("이미지를 업로드", "카메라로 촬영"))

if img_source == "이미지를 업로드":
  img_file = st.sidebar.file_uploader("이미지를 선택해 주세요.", type =["png", "jpg", "jpeg"])
elif img_source == "카메라로 촬영":
  img_file = st.camera_input("카메라로 촬영")

if img_file is not None:
  with st.spinner("측정 중..."):
    img = Image.open(img_file)
    st.image(img, caption = "대상 이미지", width = 480)
    st.write("")

    # 예측
    results = predict(img)

    # 결과 표시
    st.subheader("판정 결과")
    n_top = 3 # 확률이 높은 순으로 3위까지 반환한다
    for result in results[:n_top]:
      st.write(str(round(result[2]*100, 2)) + "%의 확률로" + result[0] + "입니다.")

    # 원 그래프 표시
    pie_labels = [result[1] for result in results[:n_top]]
    pie_labels.append("others") # 기타
    pie_probs = [result[2] for result in results[:n_top]]
    pie_probs.append(sum([result[2] for result in results[n_top:]]))

    # 기타
    fig, ax = plt.subplots()
    wedgeprops = {"width" : 0.3, "edgecolor" : "white"}
    textprops = {"fontsize" : 6}
      ax.pie(pie_probs, labels = pie_labels, counterclock = False, startangle = 90, textprops = textprops, autopct = "%.2f", wedgeprops = wedgeprops)

    # 원 그래프
    st.pyplot(fig)

st.sidebar.write("")
st.sidebar.write("")

st.sidebar.caption("""
이 앱은 Fashion-MNIST를 훈련 데이터로 사용하고 있습니다. \n
Copyright (c) 2017 Zalando SE \n
Released under the MIT license \n
https://github.com/zalandoresearch/fashion-minist#license
""")

### 이번에 사용한 주요 Streamlit 코드의 설명
- **st.title()** 함수로 타이틀을 표시함. sidebar를 끼워서 사이드 바에 표시되게 함
- **st.write()**는 다양한 타입의 인수를 화면에 표시할 수 있는 만능 함수임
- **st.radio()** 함수는 라디오 버튼을 배치함
- **st.file_uploader()** 함수에 의해 사용자가 파일을 업로드 가능한 영역이 배치됨
- **st.camera_input()** 함수로 웹 카메라가 실행되어 촬영이 가능하게 됨
- **st.image()** 함수로 화면에 이미지를 표시할 수 있음
- **st.pyplot()** 함수로 matplotlib 그래프를 표시할 수 있음
- 이외 다른 기능들은 공식 문서 참조
- Streamlit LIbrary
> URL https://docs.streamlit.io/library

## Authtoken의 설정
- ngrok로 접속하기 위해 필요한 Authtoken을 설정함
- 다음 코드에서 YourAuthtoken 부분을 자신의 ngrok의 Authtoken로 바꾸면 됨
> !ngrok authtoken YourAuthtoken
---
- 다음 절에서 GitHub을 사용하는데 실수로 자신의 AuthToken을 GitHub에 업로드하지 않도록 <font color = "red">**주의!**</font>

In [ ]:
# Authtoken의 설정
!ngrok authtoken 2hDy5JJCU1A2tTmgBQ4wJFNptZz_3Rn9Z2Gtit1feVSjiGdvM

## 앱 실행과 동작 확인
- streamlit의 run 명령어로 앱을 실행함

In [ ]:
# 앱의 실행
!streamlit run app.py &>/dev/null& # &>/dev/null&에 의해 출력을 표시하지 않고 백그라운드 작업으로 실행

- ngrok 프로세스를 종료한 다음에 새로 포트를 지정하여 접속함
- 접속 결과로 URL을 취득할 수 있음
- ngrok의 무료 요금제에서는 동시에 1개의 프로세스만 동작시킬 수 있음
- 따라서 오류가 발생했다면 **런타임 $\rightarrow$ 세션 관리**에서 불필요한 Google Colaboratory 세션을 종료하면 됨

In [ ]:
# ngrok에 의한 접속
ngrok.kill() # 프로세스 종료
url = ngrok.connect(port = "8501") # 접속

- URL을 표시하고 링크에서 앱이 동작하는지 확인함
  Google Colaboratory 서버를 이용한 일시적 공개라는 점 유의

In [ ]:
# 앱의 url을 표시
print(url)

- 링크의 URL을 복사하고 브라우저 URL란에 붙여서 http를 https로 변경하고 페이지를 표시
- 앱 화면이 표시되는 것을 확인
- 그 다음에 적당한 이미지 파일을 업로드하여 결과가 표시되는 것을 확인

## requirements.txt의 작성
- Streamlit Cloud의 서버에서 앱을 작동하기 위해서 requirements.txt를 작성해야 함
- 이 파일에서는 필요한 라이브러리의 버전을 지정함
- 우선 앱에서 import한 라이브러리의 버전을 확인함

In [ ]:
# 각 라이브러리의 버전을 확인
import streamlit
import torch
import torchvision
import PIL
import matplotlib

print("streamlit==" + streamlit.__version__)
print("torch==" + torch.__version__)
print("torchvision==" + torchvision.__version__)
print("Pillow==" + PIL.__version__)
print("matplotlib==" + matplotlib.__version__)

- 앞의 내용을 참고하여 각 라이브러리의 올바른 버전을 기술하고 requirements.txt에 저장함
- 또 Pillow와 matplotlib은 버전을 기술하지 않음

In [ ]:
# requirements.txt의 작성
with open("requirements.txt", "w") as w:
  w.write("streamlit==1.8.1\n")
  w.write("torch=2.3.0\n") # GPU 대응은 필요하지 않으므로 cu113은 기술하지 않는다
  w.write("torchvision=0.18.0\n") # # GPU 대응은 필요하지 않으므로 cu113은 기술하지 않는다
  w.write("Pillow\n")
  w.write("matplotlib\n")